# EO-Forge GCP Downloader

## EO-Forge Downloader

We will cover the basics steps to download scenes from:

-  Landsat 5
-  Landsat 8
-  Sentinel 2


In [ ]:
import os
#
from eo_forge.utils.downloader import bucket_images_downloader,gcSatImg

### Configurations

Pick dates and folders to dump the images

In [ ]:
#date
INI_DATETIME_STR='2021-08-10'
END_DATETIME_STR='2021-09-09'
#
BASE_PROD='./products-int/'
#
# IMAGES CROP
IMAGES_RAW=os.path.join(BASE_PROD,'images-raw')
os.makedirs(IMAGES_RAW,exist_ok=True)

## Sentinel 2 Tile

__NOTE__: Have you checked google cloud requisites in _eo-forge-gcp-downloader-0.ipynb_?

For Sentinel 2 you will need a boto config file.


### Query Images Data (on TILE)

In [ ]:
# Tile to Download
TILE=["19","F","CF"]
# Filters on Results
SENTINEL2_FILTERS=["*_R010_*"]

In [ ]:
import os

In [ ]:
# initImagesCheck
gcp_images=gcSatImg(spacecraft='S2')
# Check Images ond Google Cloud Sentinel Bucket
gcp_images.gcImagesCheck(TILE)
# Now filt them
gcp_images.gcImagesFilt(filters=SENTINEL2_FILTERS,dates=[INI_DATETIME_STR,END_DATETIME_STR])

In [ ]:
# we will just download one
pd_filt=gcp_images.pd_filt.iloc[[0]]

### Download Selected Images

In [ ]:
# Bands to Query
SENTINEL2_BANDS=["B02","B03","B04","B08","B11"]
# Init Google Cloud bucket Downloader
bid=bucket_images_downloader(spacecraft='S2',bands=SENTINEL2_BANDS)
# Start Downloading with up to 5 simultaneous downloads
bid.execute(pd_filt['base-url'].to_list(),arxive=IMAGES_RAW,max_proc_thread=5,force_download=False)

## Landsat 8

### Query Images Data (on PATH/ROW)

In [ ]:
# Tile to Download
TILE=["230","094"]
# Filters on Results
LANDSAT_FILTERS=["*_L1TP_*"]

In [ ]:
# initImagesCheck
gcp_images=gcSatImg(spacecraft='L8')
# Check Images ond Google Cloud Sentinel Bucket
gcp_images.gcImagesCheck(TILE)
# Now filt them
gcp_images.gcImagesFilt(filters=LANDSAT_FILTERS,dates=[INI_DATETIME_STR,END_DATETIME_STR])

In [ ]:
gcp_images.pd_filt

In [ ]:
# we will just download one
pd_filt=gcp_images.pd_filt.copy()

### Download Selected Images

In [ ]:
# Bands to Query
LANDSAT8_BANDS=["B2","B3","B4","B5","B6"]
# Init Google Cloud bucket Downloader
bid=bucket_images_downloader(spacecraft='L8',bands=LANDSAT8_BANDS)
# Start Downloading with up to 5 simultaneous downloads
bid.execute(pd_filt['base-url'].to_list(),arxive=IMAGES_RAW,max_proc_thread=5,force_download=False)

## Landsat 5 - Historic Arxive

### Query Images Data (on PATH/ROW)

In [ ]:
# Tile to Download
TILE=["230","094"]
# Filters on Results
LANDSAT_FILTERS=["*_L1TP_*"]
#date
INI_DATETIME_STR='2011-09-10'
END_DATETIME_STR='2011-11-10'

In [ ]:
# initImagesCheck
gcp_images=gcSatImg(spacecraft='L5')
# Check Images ond Google Cloud Sentinel Bucket
gcp_images.gcImagesCheck(TILE)
# Now filt them
gcp_images.gcImagesFilt(filters=LANDSAT_FILTERS,dates=[INI_DATETIME_STR,END_DATETIME_STR])

In [ ]:
gcp_images.pd_filt

In [ ]:
# we will just download one
pd_filt=gcp_images.pd_filt.iloc[[0]]

### Download Selected Images

In [ ]:
# Bands to Query
LANDSAT5_BANDS=["B1","B2","B3","B4","B5"]
# Init Google Cloud bucket Downloader
bid=bucket_images_downloader(spacecraft='L5',bands=LANDSAT5_BANDS)
# Start Downloading with up to 5 simultaneous downloads
bid.execute(pd_filt['base-url'].to_list(),arxive=IMAGES_RAW,max_proc_thread=5,force_download=False)